In [1]:
import os
import time
from rapidfuzz import fuzz
from base.Funciones import *

### Si es Excel

In [ ]:
dfRPCA = fLeerExcel(os.getcwd()+'\\base\\', 'Entidades_20251021.xlsx', 'Cruce', 1)
dfFCD = fLeerExcel(os.getcwd()+'\\base\\', 'Entidades_20251021.xlsx', 'fcd2.entidad', 0)

In [ ]:
dfRPCA = dfRPCA[(dfRPCA['Cruce Inicial'] == 'NO') & (dfRPCA['id'].isna())]

In [ ]:
cols = ['nit.1','nombre_entidad','nombre_seccional']
dfRPCA[cols] = dfRPCA[cols].apply(lambda col: col.map(rVaciosNulos))

In [ ]:
dfRPCA = dfRPCA[['fuente','nit.1','Entidad','total']].reset_index(drop=True)

### Si es BD

In [2]:
dfRPCA = RpcaEntidad(openWayu)
dfFCD = FcdEntidad(openWayu)

2025-Oct-24-11:35:25, Conectando a: 192.168.50.82



C:\Users\jlmartinez\MigrarRPCA\CruceEntidades\base\Funciones.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfS = pd.read_sql(query, conx)


2025-10-24 11:35:40.721793 DF Query Consultado from  rpca
2025-Oct-24-11:35:40, Conectando a: 192.168.50.82

2025-10-24 11:35:40.795762 DF Query Consultado from  fcd2


In [4]:
dfRPCA

,fuente,nit,nombre_entidad,seccional,ventidad,tipo,conteo
0,EXTINTAS,SIN INFORMACIÓN,HOSPITAL DEPARTAMENTAL SAN VICENTE DE PAUL - G...,HUILA,HOSPITAL DEPARTAMENTAL SAN VICENTE ALCALDIA PA...,-,1
1,DAFP,SIN INFORMACIÓN,DIRECCION SECCIONAL CESAR,,DIRECCION SECCIONAL CESAR,-,16
2,EXTINTAS,SIN INFORMACIÓN,HOSPITAL DE TURBO,ANTIOQUIA,HOSPITAL ALCALDIA TURBO,-,1
3,DAFP,SIN INFORMACIÓN,ICETEX REGIONAL SUCRE,,ICETEX REGIONAL SUCRE,-,4
4,CNSC,,EEMM,,EEMM,-,1
...,...,...,...,...,...,...,...
6503,CNSC,899999716,ASAMBLEA DE CUNDINAMARCA,,ASAMBLEA ALCALDIA CUNDINAMARCA,AS,3
6504,CNSC,890904646,ESE HOSPITAL GENERAL DE MEDELLIN,,ESE HOSPITAL GENERAL ALCALDIA MEDELLIN,-,16
6505,CNSC,,FONDO EDUCATIVO REGIONAL DE LA GUAJIRA,,FONDO EDUCATIVO REGIONAL ALCALDIA LA GUAJIRA,SE,6
6506,EXTINTAS,SIN INFORMACIÓN,MUNICIPIO DE BELTRAN,CUNDINAMARCA,ALCALDIA ALCALDIA BELTRAN,-,7


### General

In [5]:
dfFCD['nombre'] = dfFCD['nombre'].map(rLimpiar_texto).map(rRetirar_tildes)

In [6]:
#dfRPCA['Entidad'] = dfRPCA['nombre_entidad'].fillna('') +' '+ dfRPCA['nombre_seccional'].map(rLimpiaSeccional).fillna('')
dfRPCA['Entidad'] = dfRPCA['ventidad'].fillna('').map(rLimpiar_texto).map(rRetirar_tildes).map(rVaciosNulos)

In [7]:
%%time
for i, rp in enumerate(dfRPCA['Entidad']):
    print(i, rp, dfRPCA.iloc[i]['conteo'])
    dfFCD['eq'] = dfFCD['nombre'].apply(lambda x: fuzz.token_sort_ratio(rp, x))
    maximo = dfFCD.loc[dfFCD['eq'].idxmax()]
    dfRPCA.at[i,'fcd_id'] = maximo['id']
    dfRPCA.at[i,'fcd_nit'] = maximo['nit']
    dfRPCA.at[i,'fcd_nombre'] = maximo['nombre']
    dfRPCA.at[i,'eq'] = maximo['eq']
dfRPCA['simil_eq'] = rangoCalificacion(dfRPCA['eq'])

0 HOSPITAL DEPARTAMENTAL SAN VICENTE ALCALDIA PAUL - GARZON 1
1 DIRECCION SECCIONAL CESAR 16
2 HOSPITAL ALCALDIA TURBO 1
3 ICETEX REGIONAL SUCRE 4
4 EEMM 1
5 DEPARTAMENTO ADMITIVO ALCALDIA SALUD DISTRITAL -MAGDALENA 6
6 OFICINA REGISTRO INSTRUMENTOS PUBLICOS ANTIOQUIA 445
7 ALCALDIA ALCALDIA CONVENCION 2
8 POLITECNICO COLOMB. JAIME ISAZA CADAVID 283
9 DANCOOP REGIONAL VALLEDUPAR 1
10 HOSPITAL LORENCITA VILLEGAS ALCALDIA SANTOS - SAMANIEGO 7
11 INST ALCALDIA CULTURA POPULAR ALCALDIA NEIVA HUILA 6
12 ALCALDIA ALCALDIA CARMEN ALCALDIA CARUPA-CUNDINAMARCA 2
13 JUNTA ADMINISTRADORA SECCIONAL ALCALDIA DEPORTES ALCALDIA BOYACA 127
14 PEROSNERIA ALCALDIA ALCALDIA CUCUTA 3
15 EMPRESAS MUNICIPALES ALCALDIA BUGA 1
16 COLEGIO DPTAL ALCALDIA PALESTINA HUILA 3
17 ALCADIA ALCALDIA ALCALDIA DUITAMA 1
18 HOSPITAL SAN VICENTE ALCALDIA PAUL - PAEZ - 2
19 CENTRO HOSPITALARIO MATERNO INFANTIL - YOYOCO - 18
20 ALCALDIA ALCALDIA TUBARA 1
21 DANCOOP REGIONAL GUAJIRA 1
22 C. A. PADILLA 8
23 HOSPITAL ALCALDIA S

In [23]:
dfRPCA.tail()

,fuente,nit,nombre_entidad,seccional,ventidad,tipo,conteo,Entidad,fcd_id,fcd_nit,fcd_nombre,eq,simil_eq
6503,CNSC,899999716,ASAMBLEA DE CUNDINAMARCA,,ASAMBLEA ALCALDIA CUNDINAMARCA,AS,3,ASAMBLEA ALCALDIA CUNDINAMARCA,4313.0,890680107.0,ALCALDIA DE CABRERA - CUNDINAMARCA,81.250000,2.Coincide
6504,CNSC,890904646,ESE HOSPITAL GENERAL DE MEDELLIN,,ESE HOSPITAL GENERAL ALCALDIA MEDELLIN,-,16,ESE HOSPITAL GENERAL ALCALDIA MEDELLIN,6036.0,890904646.0,E.S.E. HOSPITAL GENERAL DE MEDELLIN,84.931507,2.Coincide
6505,CNSC,,FONDO EDUCATIVO REGIONAL DE LA GUAJIRA,,FONDO EDUCATIVO REGIONAL ALCALDIA LA GUAJIRA,SE,6,FONDO EDUCATIVO REGIONAL ALCALDIA LA GUAJIRA,3131.0,892115314.0,CORPORACION AUTONOMA REGIONAL DE LA GUAJIRA,66.666667,3.Revisar
6506,EXTINTAS,SIN INFORMACIÓN,MUNICIPIO DE BELTRAN,CUNDINAMARCA,ALCALDIA ALCALDIA BELTRAN,-,7,ALCALDIA ALCALDIA BELTRAN,4189.0,891901079.0,ALCALDIA DE ALCALA,79.069767,2.Coincide
6507,EXTINTAS,SIN INFORMACIÓN,INSTITUTO DE BENEFICIENCIA Y LOTERÍA DE BOYACÁ,BOYACA,INSTITUTO ALCALDIA BENEFICIENCIA Y LOTERIA ALC...,-,82,INSTITUTO ALCALDIA BENEFICIENCIA Y LOTERIA ALC...,3293.0,800103021.0,ALCALDIA DE PROVIDENCIA Y SANTA CATALINA,59.183673,3.Revisar


In [25]:
fResultadosExcel(dfRPCA, 'Entidades-migra_20251024B')

2025-Oct-24-11:40:42, Excel con los datos creado Entidades-migra_20251024B.xlsx

